In [294]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import statsmodels
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import seaborn
import dateutil
from datetime import datetime
import math
import plotly.graph_objects as go
import plotly.express as px
import math
import functions.functions as base_fx
import functions.portfolio_functions as portfolio_fx
import functions.algorithms as algos
import importlib


In [295]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)



<module 'functions.algorithms' from '../functions/algorithms.py'>

In [296]:
symbols = ['RDSA.LSE','BP.LSE']

# Get Data

In [297]:
quotes = pd.DataFrame()
#start = '2018-01-1'
start = '2010-01-4'
end = '2020-04-16'

In [298]:
for s in symbols:
    print(s)
    endpoint = f'https://eodhistoricaldata.com/api/eod/{s}?from={start}&to={end}&api_token=5d7111a684ee90.02476162&interval=d&fmt=csv'
    df = pd.read_csv(endpoint)
    #print(df.head(1))
    df = df[df['Adjusted_close'] > 0]
    #print(df.head(1))
    if len(quotes) == 0:
        quotes['Date'] = df['Date']
        quotes[s] = df['Adjusted_close']
    else:
        new_df = pd.DataFrame()
        new_df['Date'] = df['Date']
        new_df[s] = df['Adjusted_close']
        quotes = quotes.merge(new_df, how="inner", on="Date")
    #print(quotes.head(10))
#print(quotes.tail(20))

RDSA.LSE
BP.LSE


In [299]:
old_quotes = quotes
#quotes=old_quotes

#print(quotes)


In [300]:
quotes = quotes.dropna()
quotes = quotes.reset_index()
quotes['Date'] = pd.to_datetime(quotes['Date'])
quotes = quotes.set_index('Date')
#quotes = quotes[quotes.index > dateutil.parser.parse("2020-03-01")]
#print(quotes)

# Params

In [301]:
symbol1 = 'RDSA.LSE'
symbol2 = 'BP.LSE'

In [325]:
# Indicators
macd_fast_points = 1
macd_slow_points = 2
max_leverage = 2

# Could be changed
mavg_1 = 5 # default 5
mavg_2 = 40 #default 60

## Optimize trading limits looking at zscore
limit = 0.75 #default 1
exit_limit = 0.4 #default -.75
leverage_limit = 0.9

## Optimize these with bruteforce
vol_interval = 30
vol_peak_period = 365 # 1 year
vol_mean_peak_period = 60 # 2 months
coint_limit = 1
coint_period = 365


In [326]:
settings = {
    'symbol1': symbol1,
    'symbol2': symbol2,
    'mavg_1' : mavg_1,
    'mavg_2': mavg_2,
    'macd_fast_points': macd_fast_points,
    'macd_slow_points': macd_slow_points,
    'vol_interval': vol_interval,
    'vol_peak_period': vol_peak_period,
    'vol_mean_peak_period': vol_mean_peak_period,
    'limit': limit,
    'leverage_limit': leverage_limit,
    'exit_limit': exit_limit,
    'max_leverage': max_leverage,
    'coint_limit': coint_limit,
    'coint_period': coint_period
}

# Loop through each day

## Trade

In [327]:
portfolio, mavg_zscore = algos.pair_trade_with_zscore_limit(settings, quotes)

Return: 
138.4018298111118%
Vol:
0.10406969732766987


In [328]:
portfolio_fx.plot_trades_on_zscore_for_symbol(mavg_zscore, portfolio, symbol2)